In [1]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline


import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from tensorflow.keras.losses import MeanAbsolutePercentageError




2022-11-09 00:06:41.280051: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-09 00:06:41.384454: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-09 00:06:41.384469: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-09 00:06:41.401980: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-09 00:06:41.852372: W tensorflow/stream_executor/platform/de

In [2]:
train_data=pd.read_csv('../Datos/train_precios_vivienda.csv')
train_data.head()

,Unnamed: 0,id,fecha_aprobación,objeto,motivo,proposito,tipo_avaluo,tipo_credito,tipo_subsidio,departamento_inmueble,...,valor_area_construccion,area_otros,valor_area_otros,area_libre,valor_area_libre,valor_total_avaluo,valor_uvr,valor_avaluo_en_uvr,Longitud,Latitud
0,4112,5896,43090.624747,Remate,Remates,GarantÃ­a Hipotecaria,Remates,Vivienda,NaN,VALLE DEL CAUCA,...,"81968750,00",0,0,0,0,"145318750,00",2522304,"576134,95",0.000000,0.000000
1,7401,10570,NaN,OriginaciÃ³n,CrÃ©dito hipotecario de vivienda,GarantÃ­a Hipotecaria,Hipotecario,Vivienda,NaN,QUINDÃO,...,0,157,78500000,No,0,713986654,257.23250000000002,2775647.14,-75.661152,4.544027
2,10223,14600,NaN,OriginaciÃ³n,CrÃ©dito hipotecario de vivienda,GarantÃ­a Hipotecaria,Hipotecario,Vivienda,NaN,ANTIOQUIA,...,0,0,0,Si,0,270500000,259.4264,1042684.94,-75.584116,6.277020
3,4170,5967,43091.676139,OriginaciÃ³n,CrÃ©dito hipotecario de vivienda,GarantÃ­a Hipotecaria,Hipotecario,Vivienda,VIS,CUNDINAMARCA,...,0,0,0,No,0,"84840000,00",252245,"336339,67",0.000000,0.000000
4,11073,15814,NaN,OriginaciÃ³n,CrÃ©dito hipotecario de vivienda,GarantÃ­a Hipotecaria,Hipotecario,Vivienda,VIS,META,...,69306400,0,0,0,0,96346400,259.72770000000003,370951.58,-73.712370,3.565757


In [3]:
train_data['objeto'].value_counts()

OriginaciÃ³n                  8246
Originación                   2981
Remate                         343
2018-06-19 19:48:27.269-05       1
Name: objeto, dtype: int64

In [4]:
train_data['objeto'].replace('OriginaciÃ³n', 'Originación', inplace=True)
train_data['objeto'].value_counts()

Originación                   11227
Remate                          343
2018-06-19 19:48:27.269-05        1
Name: objeto, dtype: int64

In [5]:
lista_defec=[]
for i in train_data[train_data['objeto']=='2018-06-19 19:48:27.269-05'].loc[2694].values[1:]:
    lista_defec.append(i)
lista_defec.append(0.0)
train_data.loc[2694]=lista_defec

In [6]:
train_data['objeto'].replace('OriginaciÃ³n', 'Originación', inplace=True)
train_data['objeto'].value_counts()

Originación    11228
Remate           343
Name: objeto, dtype: int64

In [7]:
train_data['motivo'].replace('CrÃ©dito hipotecario de vivienda', 'Crédito hipotecario de vivienda', inplace=True)
train_data['motivo'].replace('ActualizaciÃ³n de garantÃ­as', 'Actualización de garantías', inplace=True)
train_data['motivo'].replace('DaciÃ³n en Pago', 'Dación en Pago', inplace=True)
train_data['motivo'].replace('CrÃ©dito Comercial', 'Crédito Comercial', inplace=True)

train_data['motivo'].value_counts()

Crédito hipotecario de vivienda           8551
Leasing Habitacional                       981
Empleados                                  713
Remates                                    380
Colomext Hipotecario                       312
Compra de cartera                          239
Dación en Pago                             128
Hipotecario Visto Bueno                    117
Garantía                                    62
Actualización de garantías                  62
Leasing Visto Bueno                         13
Crédito Comercial                            7
Leasing Comercial                            3
Leasing Inmobiliario - Persona Natural       1
0                                            1
Reformas                                     1
Name: motivo, dtype: int64

In [8]:
train_data['proposito'].replace('GarantÃ­a Hipotecaria', 'Garantía Hipotecaria', inplace=True)
train_data['proposito'].replace('TransacciÃ³n Comercial de Venta', 'Transacción Comercial de Venta', inplace=True)

train_data['proposito'].value_counts()

Garantía Hipotecaria              11371
Transacción Comercial de Venta      154
Valor Asegurable                     41
0                                     5
Name: proposito, dtype: int64

In [9]:
for i in train_data.columns:
    if train_data[i].dtype=='object':
        train_data[i]=train_data[i].str.replace('Ã³', 'ó')
        train_data[i]=train_data[i].str.replace('Ã¡', 'á')
        train_data[i]=train_data[i].str.replace('Ã©', 'é')
        train_data[i]=train_data[i].str.replace('Ã­', 'í')
        train_data[i]=train_data[i].str.replace('Ãº', 'ú')
        train_data[i]=train_data[i].str.replace('Ã±', 'ñ')
        train_data[i]=train_data[i].str.replace('Ã', 'í')
        train_data[i]=train_data[i].str.lower()
        


In [10]:
Lista_analisis=[]
for i in train_data.columns:
    Lista_analisis.append([i,train_data[i].dtype,train_data[i].isna().sum(),round((train_data[i].isna().sum()/len(train_data))*100,2),
                            train_data[i].nunique(),list(train_data[i].value_counts().index[:5])])
df_analisis=pd.DataFrame(Lista_analisis,columns=['Columna','Tipo','Nulos','Porcentaje de nulos','Valores únicos','Ejemplos'])
df_analisis.to_csv('../Datos/Analisis.csv',index=False)


In [11]:
df_analisis=pd.read_csv('../Datos/Analisis_2.csv')
df_analisis=df_analisis[df_analisis['Incluir']==1]
df_analisis

,Columna,Tipo,Nulos,Porcentaje de nulos,Valores únicos,Ejemplos,Incluir,Limpiar,bool
1,id,int64,0,0.0,11571,"[5896, 5469, 4953, 6551, 10501]",1.0,1.0,NaN
3,objeto,object,0,0.0,2,"['originación', 'remate']",1.0,1.0,0.0
4,motivo,object,0,0.0,16,"['crédito hipotecario de vivienda', 'leasing h...",1.0,1.0,0.0
5,proposito,object,0,0.0,4,"['garantía hipotecaria', 'transacción comercia...",1.0,1.0,0.0
6,tipo_avaluo,object,0,0.0,2,"['hipotecario', 'remates']",1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...
213,area_otros,object,0,0.0,357,"['0', '72', '60', '78', '54']",1.0,1.0,NaN
214,valor_area_otros,object,0,0.0,382,"['0', '50700000', '23760000', '18900000', '367...",1.0,1.0,NaN
215,area_libre,object,0,0.0,42,"['no', '0', 'si', '88,12', '44,35']",1.0,1.0,1.0
216,valor_area_libre,object,0,0.0,1083,"['0', '3780000', '2058000,00', '1680000', '326...",1.0,1.0,NaN


Hacemos una limpieza a varias variables que deben ser booleanas

In [12]:
train_data_2=train_data[df_analisis['Columna'].values]
lista_bool=[]
df_bool=df_analisis[df_analisis['bool']==1]
for i in df_bool['Columna']:
    temp_data=train_data_2[train_data_2[i].isin(['si','no'])]
    temp_data[i]=temp_data[i].replace('si',1)
    temp_data[i]=temp_data[i].replace('no',0)
    if temp_data.shape[0]>10000:
        print(i)
        train_data_2=temp_data
        lista_bool.append(i)
    else:
        train_data_2.drop(i,axis=1,inplace=True)
        print('Se elimina la columna',i)
    
train_data_2.shape

alcantarillado_en_el_sector
acueducto_en_el_sector
gas_en_el_sector
energia_en_el_sector
telefono_en_el_sector
vias_pavimentadas
sardineles_en_las_vias
andenes_en_las_vias
barrio_legal
Se elimina la columna demanda_interes
paradero
alumbrado
arborizacion
alcantarillado_en_el_predio
acueducto_en_el_predio
gas_en_el_predio
energia_en_el_predio
telefono_en_el_predio
Se elimina la columna tipo_inmueble
sometido_a_propiedad_horizontal
Se elimina la columna altura_permitida
predio_subdividido_fisicamente
Se elimina la columna rph
Se elimina la columna porteria
Se elimina la columna citofono
Se elimina la columna bicicletero
Se elimina la columna piscina
Se elimina la columna tanque_de_agua
Se elimina la columna club_house
Se elimina la columna garaje_visitantes
Se elimina la columna teatrino
Se elimina la columna sauna
vigilancia_privada
Se elimina la columna tipo_vigilancia
Se elimina la columna administracion
Se elimina la columna garaje_cubierto_1
Se elimina la columna garaje_doble_1
Se e

(10815, 55)

Ahora revisamos las demás variables:

In [13]:
df_limpiar=df_analisis[df_analisis['Incluir']==1]
df_limpiar=df_limpiar[df_limpiar['bool']!=1]
lista_float=[]
for i in df_limpiar['Columna']:
    if i.count('area')>0 or i.count('valor')>0:
        train_data_2[i]=train_data_2[i].str.replace(',','.')
        train_data_2[i]=train_data_2[i].str.replace('$','')
        train_data_2[i]=train_data_2[i].astype(float)
        lista_float.append(i)


In [14]:
lista_dummies=[i for i in train_data_2.columns if  i not in lista_float+lista_bool]

In [15]:
for i in lista_dummies:
    train_data_2[i]=train_data_2[i].astype('category')
    train_data_2[i]=train_data_2[i].cat.codes

In [16]:
train_data_2_bool=train_data_2[lista_bool]=train_data_2[lista_bool].astype(int)
lista_float=list(set(lista_float))
train_data_2_float=train_data_2[lista_float]=train_data_2[lista_float].astype(float)


In [17]:
train_data_2_dummies=pd.get_dummies(train_data_2,columns=lista_dummies)


In [18]:
train_df=pd.concat([train_data_2_bool,train_data_2_float,train_data_2_dummies],axis=1)
train_df[lista_float].describe()

,valor_area_terreno,valor_area_terreno,area_construccion,area_construccion,valor_area_garaje,valor_area_garaje,valor_area_construccion,valor_area_construccion,area_garaje,area_garaje,...,area_terreno,area_terreno,valor_area_deposito,valor_area_deposito,valor_area_otros,valor_area_otros,area_otros,area_otros,valor_area_privada,valor_area_privada
count,1.081500e+04,1.081500e+04,1.081500e+04,1.081500e+04,1.081500e+04,1.081500e+04,1.081500e+04,1.081500e+04,1.081500e+04,1.081500e+04,...,1.081500e+04,1.081500e+04,1.081500e+04,1.081500e+04,1.081500e+04,1.081500e+04,1.081500e+04,1.081500e+04,1.081500e+04,1.081500e+04
mean,2.199567e+07,2.199567e+07,6.416163e+04,6.416163e+04,5.768864e+07,5.768864e+07,2.482560e+08,2.482560e+08,4.872202e+05,4.872202e+05,...,8.432819e+02,8.432819e+02,3.575532e+06,3.575532e+06,3.799154e+06,3.799154e+06,1.250223e+05,1.250223e+05,2.956687e+08,2.956687e+08
std,7.417434e+07,7.417434e+07,3.942278e+06,3.942278e+06,3.365237e+09,3.365237e+09,1.035341e+10,1.035341e+10,1.569731e+07,1.569731e+07,...,5.702792e+04,5.702792e+04,2.924492e+08,2.924492e+08,3.705552e+07,3.705552e+07,6.248038e+06,6.248038e+06,6.968880e+09,6.968880e+09
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.386000e+08,1.386000e+08
75%,1.783500e+07,1.783500e+07,5.243500e+01,5.243500e+01,1.400000e+07,1.400000e+07,3.736575e+07,3.736575e+07,1.035000e+01,1.035000e+01,...,5.348000e+01,5.348000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.432781e+08,2.432781e+08
max,2.315060e+09,2.315060e+09,3.900000e+08,3.900000e+08,3.000000e+11,3.000000e+11,7.960260e+11,7.960260e+11,9.576000e+08,9.576000e+08,...,5.000000e+06,5.000000e+06,3.000000e+10,3.000000e+10,1.131750e+09,1.131750e+09,5.632000e+08,5.632000e+08,6.250000e+11,6.250000e+11


In [19]:
for i in lista_float:
    if i.count('valor_total_avaluo')==0:
        train_df[i]=train_df[i].fillna(train_df[i].mean())
        escale=StandardScaler()
        train_df[i]=escale.fit_transform(train_df[i])

In [20]:
target_df=train_df['valor_total_avaluo']
train_df.drop('valor_total_avaluo',axis=1,inplace=True)


In [21]:
X_train, X_test, y_train, y_test = train_test_split(train_df, target_df, test_size=0.2, random_state=2022)

In [22]:
import kerastuner as kt
mape = MeanAbsolutePercentageError()


def build_model(hp):
  model = tf.keras.Sequential()
  
  # Dejamos que keras tunee el número de neuronas por capa
  # Escogiendo entre 2 y 216 unidades por capa 
  hp_units1 = hp.Int('units1', min_value=2, max_value=216, step=2)
  hp_units2 = hp.Int('units2', min_value=2, max_value=216, step=2)
  hp_units3 = hp.Int('units3', min_value=2, max_value=216, step=2)
  hp_units4 = hp.Int('units3', min_value=2, max_value=216, step=2)
  hp_units5 = hp.Int('units3', min_value=2, max_value=216, step=2)
  model.add(Dense(units=hp_units1, activation='relu'))
  model.add(tf.keras.layers.Dense(units=hp_units2, activation='relu'))
  model.add(tf.keras.layers.Dense(units=hp_units3, activation='relu'))
  model.add(Dense(1, kernel_initializer='normal', activation='linear'))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
      loss=mape,
      metrics=[mape]
  )

  return model

# HyperBand algorithm from keras tuner
tuner = kt.Hyperband(
    build_model,
    objective='val_mean_absolute_percentage_error',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo'
)

tuner.search(X_train, y_train, epochs=10, validation_split=0.2)

Trial 30 Complete [00h 00m 31s]
val_mean_absolute_percentage_error: 1848236.5

Best val_mean_absolute_percentage_error So Far: 3771.52392578125
Total elapsed time: 00h 06m 28s
INFO:tensorflow:Oracle triggered exit


In [23]:
for h_param in [f"units{i}" for i in range(1,4)] + ['learning_rate']:
  print(h_param, tuner.get_best_hyperparameters()[0].get(h_param))

units1 212
units2 12
units3 154
learning_rate 0.0001


In [24]:
best_model = tuner.get_best_models()[0]
best_model.build(X_train.shape)
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (8652, 212)               5595104   
                                                                 
 dense_1 (Dense)             (8652, 12)                2556      
                                                                 
 dense_2 (Dense)             (8652, 154)               2002      
                                                                 
 dense_3 (Dense)             (8652, 1)                 155       
                                                                 
Total params: 5,599,817
Trainable params: 5,599,817
Non-trainable params: 0
_________________________________________________________________


In [25]:
best_model.fit(
    X_train, 
    y_train,
    epochs=10,
    batch_size=64
)

Epoch 1/10
136/136 [==============================] - 3s 19ms/step - loss: 3088.7527 - mean_absolute_percentage_error: 3070.8970
Epoch 2/10
136/136 [==============================] - 3s 19ms/step - loss: 3147.9136 - mean_absolute_percentage_error: 3129.7046
Epoch 3/10
136/136 [==============================] - 3s 19ms/step - loss: 4602.9023 - mean_absolute_percentage_error: 4576.0010
Epoch 4/10
136/136 [==============================] - 3s 20ms/step - loss: 3860.6633 - mean_absolute_percentage_error: 3838.1963
Epoch 5/10
136/136 [==============================] - 3s 19ms/step - loss: 3312.6760 - mean_absolute_percentage_error: 3293.4824
Epoch 6/10
136/136 [==============================] - 3s 19ms/step - loss: 5181.7222 - mean_absolute_percentage_error: 5151.3628
Epoch 7/10
136/136 [==============================] - 3s 19ms/step - loss: 3022.9124 - mean_absolute_percentage_error: 3005.4502
Epoch 8/10
136/136 [==============================] - 3s 19ms/step - loss: 2574.0437 - mean_absol

In [26]:
mape(y_test, best_model.predict(X_test)).numpy()


68/68 [==============================] - 0s 4ms/step


232.89444